# Network Scanning & Enumeration - Hands-On Lab

**Part of HackLearn Pro - Module #15**

## Learning Objectives

In this interactive lab, you will:

1. Master **TCP/UDP scanning techniques** using Nmap and Python
2. Perform **service enumeration** across SMB, SNMP, and LDAP protocols
3. Conduct **vulnerability scanning** with automated tools
4. Learn **IDS evasion techniques** and stealth scanning methods
5. Build **custom scanning tools** using Scapy for packet crafting

**MITRE ATT&CK Coverage**: T1046 (Network Service Discovery)

**Legal Notice**: These labs are for AUTHORIZED educational purposes only. Only scan systems you own or have explicit written permission to test. Unauthorized scanning is illegal under CFAA and equivalent international laws.

## Setup

Install required packages for network scanning exercises:

In [ ]:
# Install required packages
!pip install python-nmap scapy pysnmp requests beautifulsoup4

In [ ]:
import subprocess
import socket
import ipaddress
import re
from datetime import datetime
import json
try:
    import nmap
    from scapy.all import IP, TCP, sr1, ICMP
    print("All required packages imported successfully!")
except ImportError as e:
    print(f"Import error: {e}")
    print("Please install missing packages using pip")

---

## Lab 1: Nmap Scanning Techniques

Learn various TCP/UDP scanning methods and their use cases.

### TCP SYN Scan (Half-Open Scan)
The most popular scan type - stealthy and efficient.

In [ ]:
# Initialize Nmap scanner
nm = nmap.PortScanner()

# Target for legal scanning (Nmap's official test server)
target = 'scanme.nmap.org'

print(f"Starting SYN scan on {target}...")
print("This may take 30-60 seconds...\n")

# Perform SYN scan on common ports
# -sS = SYN scan, -F = Fast mode (100 most common ports)
nm.scan(target, arguments='-sS -F')

# Display results
for host in nm.all_hosts():
    print(f"Host: {host} ({nm[host].hostname()})")
    print(f"State: {nm[host].state()}")
    
    for proto in nm[host].all_protocols():
        print(f"\nProtocol: {proto}")
        ports = nm[host][proto].keys()
        for port in sorted(ports):
            state = nm[host][proto][port]['state']
            print(f"  Port {port}: {state}")

### Service Version Detection
Identify specific services and versions running on open ports.

In [ ]:
print(f"Starting service detection on {target}...")
print("This scan is more intrusive and may take longer...\n")

# -sV = Version detection
nm.scan(target, arguments='-sV -p 22,80,443')

for host in nm.all_hosts():
    for proto in nm[host].all_protocols():
        ports = nm[host][proto].keys()
        for port in sorted(ports):
            service = nm[host][proto][port]
            print(f"Port {port}/{proto}:")
            print(f"  State: {service['state']}")
            print(f"  Service: {service.get('name', 'unknown')}")
            print(f"  Version: {service.get('version', 'unknown')}")
            print(f"  Product: {service.get('product', 'unknown')}")
            print()

### UDP Scan
Scan UDP services (slower but necessary for complete reconnaissance).

In [ ]:
# UDP scan on common services
# -sU = UDP scan, targeting SNMP (161), DNS (53)
print("Starting UDP scan (this is slower than TCP)...\n")

nm.scan(target, '53,161', arguments='-sU')

for host in nm.all_hosts():
    if 'udp' in nm[host]:
        print(f"UDP ports on {host}:")
        for port in nm[host]['udp'].keys():
            state = nm[host]['udp'][port]['state']
            name = nm[host]['udp'][port].get('name', 'unknown')
            print(f"  Port {port}: {state} ({name})")
    else:
        print(f"No open UDP ports found on {host}")

### Custom TCP Scanner with Scapy
Build a basic SYN scanner from scratch to understand the mechanics.

In [ ]:
def syn_scan(target, port):
    """
    Perform a SYN scan on a single port.
    Returns: 'open', 'closed', or 'filtered'
    """
    try:
        # Craft SYN packet
        syn_packet = IP(dst=target)/TCP(dport=port, flags='S')
        
        # Send packet and wait for response (timeout=2 seconds)
        response = sr1(syn_packet, timeout=2, verbose=0)
        
        if response is None:
            return 'filtered'  # No response = filtered by firewall
        elif response.haslayer(TCP):
            if response[TCP].flags == 0x12:  # SYN-ACK flags
                # Send RST to close connection
                rst = IP(dst=target)/TCP(dport=port, flags='R')
                sr1(rst, timeout=1, verbose=0)
                return 'open'
            elif response[TCP].flags == 0x14:  # RST-ACK flags
                return 'closed'
        return 'unknown'
    except Exception as e:
        return f'error: {e}'

# Test on scanme.nmap.org
print("Custom SYN Scanner - Testing common ports\n")
test_ports = [22, 80, 443, 8080]

for port in test_ports:
    result = syn_scan(target, port)
    print(f"Port {port}: {result}")

### Scan Results Analysis
Parse and visualize scan data for reporting.

In [ ]:
# Comprehensive scan with OS detection
print("Running comprehensive scan (OS detection + service versions)...\n")

nm.scan(target, arguments='-sS -sV -O -F')

# Generate structured report
report = {
    'scan_time': datetime.now().isoformat(),
    'target': target,
    'results': {}
}

for host in nm.all_hosts():
    host_data = {
        'hostname': nm[host].hostname(),
        'state': nm[host].state(),
        'open_ports': [],
        'os_guess': []
    }
    
    # Collect open ports
    for proto in nm[host].all_protocols():
        for port in nm[host][proto].keys():
            if nm[host][proto][port]['state'] == 'open':
                host_data['open_ports'].append({
                    'port': port,
                    'protocol': proto,
                    'service': nm[host][proto][port].get('name', 'unknown'),
                    'version': nm[host][proto][port].get('version', 'unknown')
                })
    
    # OS detection results
    if 'osmatch' in nm[host]:
        for os in nm[host]['osmatch']:
            host_data['os_guess'].append({
                'name': os['name'],
                'accuracy': os['accuracy']
            })
    
    report['results'][host] = host_data

# Display formatted report
print(json.dumps(report, indent=2))
print(f"\nTotal open ports found: {sum(len(h['open_ports']) for h in report['results'].values())}")

---

## Lab 2: SMB/NetBIOS Enumeration

Enumerate Windows file sharing services (TCP ports 139, 445).

### SMB Version Detection

In [ ]:
def check_smb_ports(target):
    """
    Check if SMB ports (139, 445) are open.
    """
    smb_ports = [139, 445]
    results = {}
    
    for port in smb_ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((target, port))
        results[port] = 'open' if result == 0 else 'closed'
        sock.close()
    
    return results

# Test SMB ports on target
print("Checking SMB/NetBIOS ports...\n")
smb_status = check_smb_ports(target)

for port, status in smb_status.items():
    service = "NetBIOS-SSN" if port == 139 else "Microsoft-DS"
    print(f"Port {port} ({service}): {status}")

if 'open' in smb_status.values():
    print("\nSMB service detected! Further enumeration possible.")
else:
    print("\nNo SMB services detected on target.")

### SMB Share Enumeration (Conceptual)
Understanding how tools like enum4linux and smbclient work.

In [ ]:
# Educational example - shows the commands used for SMB enumeration
# These would be run in a real penetration test environment

smb_commands = {
    'enum4linux': [
        "enum4linux -a <target>  # All enumeration",
        "enum4linux -U <target>  # User enumeration",
        "enum4linux -S <target>  # Share enumeration",
        "enum4linux -G <target>  # Group enumeration"
    ],
    'smbclient': [
        "smbclient -L //<target> -N  # List shares (null session)",
        "smbclient //<target>/sharename -U username  # Connect to share",
        "smbclient //<target>/IPC$ -N  # Connect to IPC$ share"
    ],
    'rpcclient': [
        'rpcclient -U "" -N <target>  # Null session',
        "enumdomusers  # Enumerate domain users",
        "enumdomgroups  # Enumerate domain groups",
        "queryuser <RID>  # Get user details"
    ],
    'nmap_scripts': [
        "nmap --script smb-enum-shares -p 445 <target>",
        "nmap --script smb-enum-users -p 445 <target>",
        "nmap --script smb-os-discovery -p 445 <target>",
        "nmap --script smb-vuln* -p 445 <target>  # Check vulnerabilities"
    ]
}

print("SMB Enumeration Command Reference\n")
print("=" * 60)

for tool, commands in smb_commands.items():
    print(f"\n{tool.upper()}:")
    for cmd in commands:
        print(f"  {cmd}")

print("\n" + "=" * 60)
print("\nNote: These commands require appropriate tools installed")
print("and should only be used on authorized systems.")

### SMB Vulnerability Scanner
Check for common SMB vulnerabilities using Nmap scripts.

In [ ]:
# Use Nmap NSE scripts to check for SMB vulnerabilities
print("Scanning for SMB vulnerabilities...\n")

# Check if SMB ports are open first
nm.scan(target, '445', arguments='-sV')

if '445' in nm[target].get('tcp', {}):
    if nm[target]['tcp']['445']['state'] == 'open':
        print("Port 445 is open. Running vulnerability scripts...\n")
        
        # Run SMB vulnerability scripts
        nm.scan(target, '445', arguments='--script smb-vuln*')
        
        # Display results
        if 'hostscript' in nm[target]:
            for script in nm[target]['hostscript']:
                print(f"Script: {script['id']}")
                print(f"Output: {script['output']}")
                print("-" * 60)
        else:
            print("No vulnerabilities detected or scripts not available.")
    else:
        print("Port 445 is closed.")
else:
    print("Port 445 not found in scan results.")

print("\nCommon SMB Vulnerabilities:")
print("  - MS17-010 (EternalBlue): Remote code execution")
print("  - MS08-067: Remote code execution")
print("  - Null session enumeration")
print("  - SMB signing not required")

### NetBIOS Name Resolution
Query NetBIOS names and workgroup information.

In [ ]:
def parse_netbios_name(name_bytes):
    """
    Parse NetBIOS name from response bytes.
    Educational demonstration of NetBIOS protocol.
    """
    try:
        # NetBIOS names are 16 bytes, padded with spaces
        name = name_bytes[:15].decode('ascii').strip()
        suffix = name_bytes[15]
        return name, suffix
    except:
        return None, None

# NetBIOS suffix meanings
netbios_suffixes = {
    0x00: "Workstation Service",
    0x03: "Messenger Service",
    0x20: "File Server Service",
    0x1B: "Domain Master Browser",
    0x1C: "Domain Controllers",
    0x1D: "Master Browser",
    0x1E: "Browser Service Elections"
}

print("NetBIOS Name Service (NBNS) Overview\n")
print("Port: UDP 137")
print("Purpose: Name resolution for Windows networks\n")

print("Common NetBIOS Suffixes:")
for suffix, description in netbios_suffixes.items():
    print(f"  <{hex(suffix)[2:].upper().zfill(2)}>: {description}")

print("\nEnumeration Command:")
print("  nbtscan <target>  # Scan for NetBIOS names")
print("  nmblookup -A <target>  # Query NetBIOS names")

---

## Lab 3: SNMP Enumeration

Query SNMP services (UDP 161) for system information.

### SNMP Version Detection

In [ ]:
def check_snmp(target, community='public'):
    """
    Check if SNMP is accessible with given community string.
    """
    try:
        # Test UDP 161 connectivity
        sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        sock.settimeout(2)
        
        # SNMP v1 GET request for sysDescr (simplified)
        # In production, use pysnmp library for proper SNMP communication
        
        sock.close()
        return True
    except:
        return False

# SNMP community strings to test (educational)
common_communities = ['public', 'private', 'community', 'snmp']

print("SNMP Enumeration Overview\n")
print("Protocol: UDP port 161 (queries), 162 (traps)")
print("Versions: SNMPv1, SNMPv2c (community-based), SNMPv3 (secure)\n")

print("Common Default Community Strings:")
for community in common_communities:
    print(f"  - {community}")

print("\nSecurity Risks:")
print("  - Default community strings expose sensitive data")
print("  - SNMPv1/v2c transmit in cleartext")
print("  - Can enumerate users, processes, network config")
print("  - Write access (private) allows configuration changes")

### SNMP MIB Walking (Conceptual)
Understanding Management Information Base (MIB) queries.

In [ ]:
# Common SNMP OIDs (Object Identifiers)
snmp_oids = {
    'system': {
        '1.3.6.1.2.1.1.1.0': 'sysDescr (System Description)',
        '1.3.6.1.2.1.1.3.0': 'sysUpTime',
        '1.3.6.1.2.1.1.4.0': 'sysContact',
        '1.3.6.1.2.1.1.5.0': 'sysName (Hostname)',
        '1.3.6.1.2.1.1.6.0': 'sysLocation'
    },
    'network': {
        '1.3.6.1.2.1.2.2.1.2': 'ifDescr (Interface Description)',
        '1.3.6.1.2.1.4.20.1.1': 'ipAdEntAddr (IP Addresses)',
        '1.3.6.1.2.1.4.21.1.1': 'ipRouteDest (Routing Table)',
        '1.3.6.1.2.1.6.13.1.3': 'tcpConnLocalPort (TCP Connections)'
    },
    'processes': {
        '1.3.6.1.2.1.25.4.2.1.2': 'hrSWRunName (Running Processes)',
        '1.3.6.1.2.1.25.4.2.1.4': 'hrSWRunPath (Process Paths)',
        '1.3.6.1.2.1.25.4.2.1.5': 'hrSWRunParameters (Process Args)'
    },
    'storage': {
        '1.3.6.1.2.1.25.2.3.1.3': 'hrStorageDescr (Storage Description)',
        '1.3.6.1.2.1.25.2.3.1.5': 'hrStorageSize (Storage Size)',
        '1.3.6.1.2.1.25.2.3.1.6': 'hrStorageUsed (Storage Used)'
    }
}

print("SNMP MIB (Management Information Base) Reference\n")
print("=" * 70)

for category, oids in snmp_oids.items():
    print(f"\n{category.upper()} INFORMATION:")
    for oid, description in oids.items():
        print(f"  {oid}")
        print(f"    -> {description}")

print("\n" + "=" * 70)
print("\nSNMP Enumeration Commands:")
print("  snmpwalk -v 2c -c public <target> 1.3.6.1.2.1.1")
print("  snmpget -v 2c -c public <target> 1.3.6.1.2.1.1.5.0")
print("  onesixtyone -c communities.txt <target>  # Bruteforce community strings")

### SNMP Enumeration Script
Simulate SNMP data extraction for reconnaissance.

In [ ]:
class SNMPEnumerator:
    """
    Educational SNMP enumeration class.
    Demonstrates data that can be extracted via SNMP.
    """
    
    def __init__(self, target, community='public'):
        self.target = target
        self.community = community
        self.results = {}
    
    def enumerate_system_info(self):
        """Query basic system information."""
        print(f"[*] Enumerating system info from {self.target}...")
        # In real implementation, use pysnmp to query OIDs
        self.results['system'] = {
            'description': 'Example: Linux server 5.10.0',
            'hostname': 'example-host',
            'uptime': '45 days, 3 hours',
            'contact': 'admin@example.com',
            'location': 'Datacenter A'
        }
        return self.results['system']
    
    def enumerate_network_info(self):
        """Query network configuration."""
        print(f"[*] Enumerating network interfaces...")
        self.results['network'] = {
            'interfaces': ['eth0', 'eth1', 'lo'],
            'ip_addresses': ['192.168.1.10', '10.0.0.5'],
            'routing_table': [
                {'dest': '0.0.0.0', 'gateway': '192.168.1.1'},
                {'dest': '10.0.0.0/8', 'gateway': '10.0.0.1'}
            ]
        }
        return self.results['network']
    
    def enumerate_processes(self):
        """Query running processes."""
        print(f"[*] Enumerating running processes...")
        self.results['processes'] = [
            {'name': 'apache2', 'path': '/usr/sbin/apache2'},
            {'name': 'mysqld', 'path': '/usr/sbin/mysqld'},
            {'name': 'sshd', 'path': '/usr/sbin/sshd'}
        ]
        return self.results['processes']
    
    def enumerate_users(self):
        """Query user accounts."""
        print(f"[*] Enumerating user accounts...")
        self.results['users'] = ['root', 'admin', 'webuser', 'backup']
        return self.results['users']
    
    def generate_report(self):
        """Generate enumeration report."""
        print("\n" + "=" * 60)
        print("SNMP ENUMERATION REPORT")
        print("=" * 60)
        print(f"\nTarget: {self.target}")
        print(f"Community String: {self.community}")
        print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("\n" + json.dumps(self.results, indent=2))

# Demonstration
enumerator = SNMPEnumerator('example.com', 'public')
enumerator.enumerate_system_info()
enumerator.enumerate_network_info()
enumerator.enumerate_processes()
enumerator.enumerate_users()
enumerator.generate_report()

print("\n[!] Defense: Use SNMPv3 with encryption, change default communities!")

---

## Lab 4: Vulnerability Scanning

Automated detection of security vulnerabilities.

### Nmap Vulnerability Scripts

In [ ]:
# List available Nmap vulnerability scripts
vuln_script_categories = {
    'SMB Vulnerabilities': [
        'smb-vuln-ms17-010',  # EternalBlue
        'smb-vuln-ms08-067',  # Conficker
        'smb-vuln-cve2009-3103',
        'smb-vuln-ms10-061'
    ],
    'Web Vulnerabilities': [
        'http-vuln-cve2017-5638',  # Apache Struts
        'http-vuln-cve2014-3704',  # Drupal SQL injection
        'http-shellshock',
        'http-slowloris-check'
    ],
    'SSL/TLS Vulnerabilities': [
        'ssl-heartbleed',
        'ssl-poodle',
        'ssl-ccs-injection',
        'ssl-dh-params'
    ],
    'Database Vulnerabilities': [
        'mysql-vuln-cve2012-2122',
        'mongodb-databases',
        'redis-info'
    ]
}

print("Nmap NSE Vulnerability Scripts\n")
print("=" * 60)

for category, scripts in vuln_script_categories.items():
    print(f"\n{category}:")
    for script in scripts:
        print(f"  - {script}")

print("\n" + "=" * 60)
print("\nUsage Examples:")
print("  nmap --script vuln <target>  # Run all vuln scripts")
print("  nmap --script smb-vuln* <target>  # SMB vulnerabilities only")
print("  nmap --script http-vuln* -p 80,443 <target>  # Web vulns")
print("\nNote: ~600 NSE scripts available, 100+ for vulnerability detection")

### Web Application Scanning
Detect common web vulnerabilities.

In [ ]:
# Web vulnerability scanning with Nmap
print("Scanning for web application vulnerabilities...\n")

# Scan web ports for vulnerabilities
nm.scan(target, '80,443', arguments='--script http-enum,http-headers,http-methods')

for host in nm.all_hosts():
    for proto in nm[host].all_protocols():
        for port in nm[host][proto].keys():
            if nm[host][proto][port]['state'] == 'open':
                print(f"\nPort {port}/{proto}:")
                print(f"  Service: {nm[host][proto][port].get('name', 'unknown')}")
                
                # Check for script results
                if 'script' in nm[host][proto][port]:
                    for script_name, output in nm[host][proto][port]['script'].items():
                        print(f"\n  Script: {script_name}")
                        print(f"  Output:\n{output}")

print("\n" + "=" * 60)
print("\nCommon Web Vulnerabilities to Check:")
print("  - Directory listing enabled")
print("  - Sensitive files exposed (.git, .env, backup files)")
print("  - Unnecessary HTTP methods (PUT, DELETE, TRACE)")
print("  - Missing security headers (CSP, X-Frame-Options)")
print("  - Outdated software versions")

### SSL/TLS Vulnerability Scanning
Check for cryptographic weaknesses.

In [ ]:
# SSL/TLS vulnerability scanning
ssl_scripts = [
    'ssl-heartbleed',
    'ssl-poodle',
    'ssl-ccs-injection',
    'ssl-cert',
    'ssl-enum-ciphers'
]

print("SSL/TLS Vulnerability Assessment\n")

# Check if HTTPS is available
nm.scan(target, '443')

if '443' in nm[target].get('tcp', {}):
    if nm[target]['tcp']['443']['state'] == 'open':
        print("Port 443 open. Running SSL/TLS checks...\n")
        
        # Run SSL scripts
        script_args = ','.join(ssl_scripts)
        nm.scan(target, '443', arguments=f'--script {script_args}')
        
        # Display results
        if 'script' in nm[target]['tcp']['443']:
            for script_name, output in nm[target]['tcp']['443']['script'].items():
                print(f"Script: {script_name}")
                print(f"Result:\n{output}")
                print("-" * 60)
    else:
        print("Port 443 closed - no HTTPS service detected.")
else:
    print("Port 443 not in scan results.")

print("\nCritical SSL/TLS Vulnerabilities:")
print("  - Heartbleed (CVE-2014-0160): Memory disclosure")
print("  - POODLE (CVE-2014-3566): SSLv3 padding oracle")
print("  - BEAST (CVE-2011-3389): CBC cipher attack")
print("  - Weak ciphers: RC4, DES, export-grade")
print("  - Expired/invalid certificates")

### Custom Vulnerability Checker
Build a simple vulnerability assessment tool.

In [ ]:
class VulnerabilityScanner:
    """
    Basic vulnerability scanner combining multiple checks.
    """
    
    def __init__(self, target):
        self.target = target
        self.vulnerabilities = []
        self.nm = nmap.PortScanner()
    
    def check_open_ports(self):
        """Identify unnecessarily open ports."""
        print("[*] Checking for open ports...")
        risky_ports = {
            21: 'FTP - Unencrypted file transfer',
            23: 'Telnet - Unencrypted remote access',
            139: 'NetBIOS - Information disclosure',
            445: 'SMB - File sharing vulnerabilities',
            3389: 'RDP - Remote desktop vulnerabilities',
            5900: 'VNC - Remote desktop vulnerabilities'
        }
        
        self.nm.scan(self.target, arguments='-F')
        
        for host in self.nm.all_hosts():
            for proto in self.nm[host].all_protocols():
                for port in self.nm[host][proto].keys():
                    if port in risky_ports and self.nm[host][proto][port]['state'] == 'open':
                        self.vulnerabilities.append({
                            'severity': 'HIGH',
                            'category': 'Open Port',
                            'description': f"Port {port} open: {risky_ports[port]}"
                        })
    
    def check_outdated_services(self):
        """Detect potentially outdated service versions."""
        print("[*] Checking service versions...")
        
        self.nm.scan(self.target, arguments='-sV -F')
        
        for host in self.nm.all_hosts():
            for proto in self.nm[host].all_protocols():
                for port in self.nm[host][proto].keys():
                    service = self.nm[host][proto][port]
                    if 'version' in service and service['version']:
                        # Check for known vulnerable versions (simplified)
                        version = service['version']
                        if service['name'] == 'ssh' and version.startswith('OpenSSH_7.'):
                            self.vulnerabilities.append({
                                'severity': 'MEDIUM',
                                'category': 'Outdated Software',
                                'description': f"Port {port}: Potentially outdated {service['name']} {version}"
                            })
    
    def generate_report(self):
        """Generate vulnerability assessment report."""
        print("\n" + "=" * 60)
        print("VULNERABILITY ASSESSMENT REPORT")
        print("=" * 60)
        print(f"\nTarget: {self.target}")
        print(f"Scan Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"\nTotal Vulnerabilities Found: {len(self.vulnerabilities)}")
        
        if self.vulnerabilities:
            severity_counts = {'HIGH': 0, 'MEDIUM': 0, 'LOW': 0}
            for vuln in self.vulnerabilities:
                severity_counts[vuln['severity']] += 1
            
            print(f"  - HIGH: {severity_counts['HIGH']}")
            print(f"  - MEDIUM: {severity_counts['MEDIUM']}")
            print(f"  - LOW: {severity_counts['LOW']}")
            
            print("\nDetailed Findings:\n")
            for i, vuln in enumerate(self.vulnerabilities, 1):
                print(f"{i}. [{vuln['severity']}] {vuln['category']}")
                print(f"   {vuln['description']}\n")
        else:
            print("\nNo vulnerabilities detected in this scan.")
        
        print("=" * 60)

# Run vulnerability scan
scanner = VulnerabilityScanner(target)
scanner.check_open_ports()
scanner.check_outdated_services()
scanner.generate_report()

---

## Lab 5: IDS Evasion Techniques

Learn stealth scanning methods to avoid detection.

### Timing and Performance Tuning

In [ ]:
# Nmap timing templates
timing_templates = {
    'T0': {
        'name': 'Paranoid',
        'speed': '1 port every 5 minutes',
        'use_case': 'Extreme stealth, evade IDS',
        'detection_risk': 'Very Low'
    },
    'T1': {
        'name': 'Sneaky',
        'speed': '15 seconds between probes',
        'use_case': 'Slow scan, avoid IDS',
        'detection_risk': 'Low'
    },
    'T2': {
        'name': 'Polite',
        'speed': '0.4 seconds between probes',
        'use_case': 'Reduce network load',
        'detection_risk': 'Medium-Low'
    },
    'T3': {
        'name': 'Normal',
        'speed': 'Default timing',
        'use_case': 'Default, balanced',
        'detection_risk': 'Medium'
    },
    'T4': {
        'name': 'Aggressive',
        'speed': 'Fast scan, 5 min timeout',
        'use_case': 'Fast networks, pentesting',
        'detection_risk': 'High'
    },
    'T5': {
        'name': 'Insane',
        'speed': 'Maximum speed, 75s timeout',
        'use_case': 'Very fast networks only',
        'detection_risk': 'Very High'
    }
}

print("Nmap Timing Templates for IDS Evasion\n")
print("=" * 70)

for template, info in timing_templates.items():
    print(f"\n{template}: {info['name']}")
    print(f"  Speed: {info['speed']}")
    print(f"  Use Case: {info['use_case']}")
    print(f"  Detection Risk: {info['detection_risk']}")

print("\n" + "=" * 70)
print("\nUsage:")
print("  nmap -T0 <target>  # Slowest, most stealthy")
print("  nmap -T4 <target>  # Fast, typical for pentesting")
print("\nCustom Timing:")
print("  --scan-delay 10s   # 10 second delay between probes")
print("  --max-rate 100     # Max 100 packets per second")

### Packet Fragmentation
Split packets to evade signature-based IDS.

In [ ]:
# Fragmentation techniques
fragmentation_methods = {
    'IP Fragmentation (-f)': {
        'description': 'Fragment IP packets into 8-byte chunks',
        'command': 'nmap -f <target>',
        'effectiveness': 'Bypasses simple packet filters',
        'limitation': 'Modern IDS reassemble fragments'
    },
    'Custom MTU (--mtu)': {
        'description': 'Set custom Maximum Transmission Unit',
        'command': 'nmap --mtu 16 <target>',
        'effectiveness': 'More control than -f flag',
        'limitation': 'MTU must be multiple of 8'
    },
    'Data Length (--data-length)': {
        'description': 'Append random data to packets',
        'command': 'nmap --data-length 25 <target>',
        'effectiveness': 'Evade packet size filters',
        'limitation': 'Increases bandwidth usage'
    }
}

print("Packet Fragmentation Techniques\n")
print("=" * 70)

for method, details in fragmentation_methods.items():
    print(f"\n{method}:")
    print(f"  Description: {details['description']}")
    print(f"  Command: {details['command']}")
    print(f"  Effectiveness: {details['effectiveness']}")
    print(f"  Limitation: {details['limitation']}")

print("\n" + "=" * 70)
print("\nExample: Fragmented SYN Scan")
print("  nmap -f -sS -p 1-1000 <target>")
print("\nDefense: Enable fragment reassembly on IDS/firewall")

### Decoy Scanning
Use decoy IP addresses to obscure scan source.

In [ ]:
# Decoy scanning demonstration
print("Decoy Scanning Techniques\n")
print("=" * 60)

print("\nConcept:")
print("  Send scan packets from multiple IP addresses simultaneously")
print("  Target sees traffic from real scanner + decoy IPs")
print("  Difficult to identify true attacker in logs")

print("\nDecoy Options:")
print("  1. Manual decoy IPs:")
print("     nmap -D 192.168.1.5,192.168.1.6,ME <target>")
print("     (ME = your real IP position)")

print("\n  2. Random decoys:")
print("     nmap -D RND:10 <target>")
print("     (Generate 10 random decoy IPs)")

print("\n  3. Specific position:")
print("     nmap -D decoy1,decoy2,ME,decoy3,decoy4 <target>")
print("     (Place real IP in middle of decoys)")

print("\nEffectiveness:")
print("  + Obscures true source in packet captures")
print("  + Increases IDS log noise")
print("  - Decoys must be 'up' hosts (send RST responses)")
print("  - Advanced IDS can correlate patterns")
print("  - Doesn't hide source in SYN-ACK responses")

print("\nLegal Warning:")
print("  Using decoy IPs may implicate innocent parties!")
print("  Only use on authorized penetration tests with caution.")

# Demonstrate decoy packet structure (conceptual)
print("\n" + "=" * 60)
print("\nTarget sees multiple SYN packets:")
decoy_ips = ['192.168.1.100', '192.168.1.101', '192.168.1.102 (YOU)', '192.168.1.103']
for i, ip in enumerate(decoy_ips, 1):
    print(f"  {i}. SYN packet from {ip} -> target:80")
print("\nChallenge: Which one is the real attacker?")

### Source Port Spoofing
Use trusted source ports to bypass firewall rules.

In [ ]:
# Source port manipulation
trusted_ports = {
    53: 'DNS - Often allowed through firewalls',
    80: 'HTTP - Commonly allowed outbound',
    443: 'HTTPS - Commonly allowed outbound',
    20: 'FTP-DATA - Active FTP connections',
    88: 'Kerberos - Domain authentication'
}

print("Source Port Spoofing for Firewall Evasion\n")
print("=" * 60)

print("\nConcept:")
print("  Some firewalls allow traffic from 'trusted' source ports")
print("  Example: Allow DNS responses from source port 53")
print("  Attacker sends scan from port 53 to bypass filter")

print("\nCommonly Trusted Source Ports:")
for port, description in trusted_ports.items():
    print(f"  Port {port}: {description}")

print("\nNmap Source Port Options:")
print("  -g 53 or --source-port 53")
print("  Example: nmap -g 53 -p 1-1000 <target>")

print("\nEffectiveness:")
print("  + Bypasses poorly configured firewalls")
print("  + Works on stateless packet filters")
print("  - Ineffective against stateful firewalls")
print("  - Modern firewalls inspect full connection state")

print("\nReal-World Example:")
print("  Firewall rule: 'Allow UDP from any:53 to internal:any'")
print("  Intent: Allow DNS responses")
print("  Exploit: Attacker scans from source port 53")
print("  Result: Scan packets bypass firewall")

print("\nDefense:")
print("  - Use stateful firewalls that track connections")
print("  - Implement egress filtering (outbound rules)")
print("  - Don't trust source port alone for authentication")

### Idle Scan (Zombie Scan)
Advanced technique using an intermediary 'zombie' host.

In [ ]:
print("Idle Scan (Zombie Scan) - Advanced IDS Evasion\n")
print("=" * 60)

print("\nConcept:")
print("  1. Find 'zombie' host with predictable IP ID sequence")
print("  2. Spoof zombie's IP address when scanning target")
print("  3. Target sends responses to zombie (not you)")
print("  4. Observe zombie's IP ID to infer port state")
print("  5. Your IP never appears in target's logs!")

print("\nHow It Works:")
print("  Step 1: Probe zombie, record IP ID (e.g., 1000)")
print("  Step 2: Send spoofed SYN to target from zombie's IP")
print("  Step 3a: If port OPEN - target sends SYN-ACK to zombie")
print("           Zombie sends RST (IP ID increments to 1001)")
print("  Step 3b: If port CLOSED - target sends RST to zombie")
print("           Zombie ignores (IP ID stays 1000)")
print("  Step 4: Probe zombie again, check IP ID")
print("  Step 5: IP ID increased by 2? Port is OPEN")
print("          IP ID increased by 1? Port is CLOSED")

print("\nNmap Idle Scan:")
print("  nmap -sI <zombie_host> <target>")
print("  Example: nmap -sI 192.168.1.50 -p 80,443 scanme.nmap.org")

print("\nRequirements for Zombie Host:")
print("  1. Predictable/incremental IP ID sequence")
print("  2. Low network activity (no other traffic)")
print("  3. Up and responding to pings/probes")
print("  4. Not behind NAT or firewall that modifies IP IDs")

print("\nGood Zombie Candidates:")
print("  - Idle printers or network devices")
print("  - IoT devices (cameras, sensors)")
print("  - Old operating systems (Windows XP, old Linux)")

print("\nAdvantages:")
print("  + Complete stealth - your IP never contacts target")
print("  + Bypass firewall rules based on source IP")
print("  + Target logs show zombie IP, not yours")

print("\nLimitations:")
print("  - Finding suitable zombie is difficult")
print("  - Modern OS use randomized IP IDs")
print("  - Very slow scan (multiple probes per port)")
print("  - Some firewalls detect spoofed packets")

print("\nDefense:")
print("  - Use randomized IP ID sequences (RFC 6864)")
print("  - Implement ingress/egress filtering (BCP 38)")
print("  - Monitor for unusual RST packet patterns")

# Find potential zombie hosts
print("\n" + "=" * 60)
print("\nFinding Zombie Hosts:")
print("  nmap --script ipidseq <target_range>")
print("  Look for 'Incremental' IP ID sequence")

---

## Summary: Key Metrics & Takeaways

### Lab Performance Metrics

In [ ]:
# Summary of network scanning capabilities
summary = {
    'scanning_techniques': {
        'TCP': ['SYN (half-open)', 'Connect', 'ACK', 'FIN/NULL/XMAS'],
        'UDP': ['UDP scan', 'SNMP enumeration'],
        'Specialized': ['Idle scan', 'Custom Scapy packets']
    },
    'enumeration_protocols': {
        'SMB/NetBIOS': 'TCP 139, 445 - Windows file sharing',
        'SNMP': 'UDP 161 - Network device management',
        'LDAP': 'TCP 389, 636 - Directory services'
    },
    'vulnerability_scanning': {
        'Nmap NSE Scripts': '600+ scripts, 100+ for vulnerabilities',
        'Categories': ['SMB', 'Web', 'SSL/TLS', 'Database'],
        'Notable': ['Heartbleed', 'EternalBlue', 'Shellshock']
    },
    'ids_evasion': {
        'Timing': 'T0 (paranoid) to T5 (insane)',
        'Fragmentation': 'IP fragments, custom MTU',
        'Obfuscation': 'Decoy IPs, source port spoofing',
        'Advanced': 'Idle/zombie scan'
    },
    'tools_covered': {
        'Scanning': 'Nmap, python-nmap, Scapy',
        'Enumeration': 'enum4linux, smbclient, snmpwalk',
        'Custom': 'Python-based scanners'
    },
    'mitre_attack': {
        'Technique': 'T1046 - Network Service Discovery',
        'Tactic': 'Discovery',
        'Usage': '90% of APT groups use network scanning'
    }
}

print("NETWORK SCANNING & ENUMERATION - LAB SUMMARY")
print("=" * 70)
print("\nTechniques Mastered:")
print(json.dumps(summary, indent=2))

print("\n" + "=" * 70)
print("\nKey Takeaways:")
print("\n1. TCP SYN Scan:")
print("   - Most popular scan type (fast, stealthy, reliable)")
print("   - Requires root/admin privileges")
print("   - Detectable by modern IDS/IPS")

print("\n2. Service Enumeration:")
print("   - SMB/NetBIOS: User lists, shares, domain info")
print("   - SNMP: System config, network topology, running processes")
print("   - Critical for lateral movement in penetration tests")

print("\n3. Vulnerability Scanning:")
print("   - Nmap NSE scripts automate vulnerability detection")
print("   - Focus on SMB (EternalBlue), SSL/TLS, web applications")
print("   - Always verify findings manually (reduce false positives)")

print("\n4. IDS Evasion:")
print("   - Timing (T0-T2): Slow scans avoid rate-based detection")
print("   - Fragmentation: Bypasses simple signature matching")
print("   - Decoy scanning: Obscures true attacker in logs")
print("   - Idle scan: Ultimate stealth (but hard to execute)")

print("\n5. Defense Strategies:")
print("   - Deploy stateful firewalls with egress filtering")
print("   - Enable IDS/IPS with fragment reassembly")
print("   - Minimize attack surface (close unnecessary ports)")
print("   - Regular vulnerability scanning of your own systems")
print("   - Monitor for scanning activity (SYN floods, probe patterns)")

print("\n" + "=" * 70)
print("\nReal-World Impact:")
print("  - Equifax (2017): Scanner found Apache Struts vuln, led to breach")
print("  - Colonial Pipeline (2021): Network enumeration enabled ransomware")
print("  - NotPetya (2017): SMB scanning spread worm globally ($10B damage)")

print("\n" + "=" * 70)
print("\nEthical Reminder:")
print("  Network scanning without authorization is ILLEGAL.")
print("  CFAA (US), Computer Misuse Act (UK), and similar laws worldwide.")
print("  Always obtain written permission before scanning systems.")
print("\n  Use these skills for:")
print("    - Authorized penetration testing")
print("    - Your own networks and systems")
print("    - Educational labs (like scanme.nmap.org)")
print("    - Bug bounty programs with explicit scope")

print("\n" + "=" * 70)
print("\nNext Steps:")
print("  1. Practice on legal targets (TryHackMe, HackTheBox, VulnHub)")
print("  2. Build your own vulnerable lab with Metasploitable")
print("  3. Learn Wireshark to analyze scan traffic")
print("  4. Study IDS/IPS rules (Snort, Suricata)")
print("  5. Explore MITRE ATT&CK Discovery tactics")

print("\n" + "=" * 70)
print("\nCongratulations! You've completed the Network Scanning lab.")
print("Continue to Module #16: Password Cracking & Credential Attacks")
print("=" * 70)